In [94]:
from fenics import * 
import matplotlib.pyplot as plt
import numpy as np
import time

#suppress some output
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)

In [95]:
## Simulation and mesh parameters

# mesh size
Lx = 1.0         
Ly = 1.0

# set grid size
mx = 20
my = int(mx*Ly)

# xi (kpar) slice
J = 31
#SHIFT = np.linspace(0.0,1.0,J) - 1.0/3.0
SHIFT = np.linspace(0.0,1.0,J)

# kperp slice
kpoints = 51;
LAMBDA = np.linspace(-1/2,1/2,kpoints);

# number of eigenvalues
evals = 2*2*int(Lx*Ly)
E = np.zeros((kpoints,J,evals))

In [96]:
## Mesh

# set up honeycomb lattice
q = 4*np.pi/(sqrt(3))
k1 = q*np.array([1,sqrt(3)])/2.0
k2 = q*np.array([1,-sqrt(3)])/2.0
k3 = -(k1+k2)
K = (k1-k2)/3

v1_h = np.array([sqrt(3),1])/2.0
v2_h = np.array([sqrt(3),-1])/2.0

# different edges
a1 = 1; b1 = 1; a2 = 0; b2 = 1;     # armchair
# a1, b1, a2, b2 = 1, 0, 0, 1       # zigzag
# a1 = 3; b1 = 2; a2 = 1; b2 = 1;     # general
# a1 = 2; b1 = 1; a2 = 1; b2 = 1;     # general

v1 = a1*v1_h + b1*v2_h
v2 = a2*v1_h + b2*v2_h

ktilde = -b1*k1+a1*k2         # general
kpar = b2*k1 - a2*k2;

# quasimomentum slice
quasimomentum = np.linspace(0.0,0.5,2)
boundary_0 = K[0]*np.ones((1,2)) + ktilde[0]*quasimomentum
boundary_1 = K[1]*np.ones((1,2)) + ktilde[1]*quasimomentum
boundary = np.concatenate((boundary_0,boundary_1),axis=0)

# create mesh      
def honeycomb_cylinder(x, y):
    return [v1[0]*x + v2[0]*y, v1[1]*x + v2[1]*y]

mesh = RectangleMesh(Point(-Lx/2,-Ly/2),Point(Lx/2,Ly/2),mx,my)#, 'left') # “left”, “right”, “left/right”, “crossed”
x = mesh.coordinates()[:,0]
y = mesh.coordinates()[:,1]

x_hat, y_hat = honeycomb_cylinder(x, y)
xy_hat_coor = np.array([x_hat, y_hat]).transpose()
mesh.coordinates()[:] = xy_hat_coor
#plot(mesh), interactive()

In [97]:
## Boundary Conditions

# sub domain for Dirichlet boundary condition
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv
        return bool( ( near(y_r, -Ly/2.0) or near(y_r, Ly/2.0) ) and on_boundary)
    
# sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        return bool( ( near(x_r, -Lx/2.0) and near(x_r, -Lx/2.0) ) and on_boundary)

    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        x_h = x_r - (Lx - 0)
        y_h = y_r
        
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h
        
## perfectly periodic problem

# sub domain for Periodic boundary condition
class PerfectlyPeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        return bool( (near(x_r, -Lx/2.0) or near(y_r, -Ly/2.0)) and 
                    (not ((near(x_r, -Lx/2.0) and near(y_r, +Ly/2.0)) or 
                            (near(x_r, +Lx/2.0) and near(y_r, -Ly/2.0))) ) and on_boundary)
        
    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        if near(x_r, +Lx/2.0) and near(y_r, +Ly/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r - Ly - 0.0
        elif near(x_r, +Lx/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r
        else:   # near(y_r, +Ly/2.0)
            x_h = x_r 
            y_h = y_r - Ly - 0.0
            
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h

In [98]:
## Finite Elements

# periodic = PeriodicBoundary()
periodic = PerfectlyPeriodicBoundary()

P2 = FiniteElement("P", triangle, 2)
element = MixedElement([P2, P2])
W = FunctionSpace(mesh, element, constrained_domain=periodic)

# define functions
(u_r, u_i) = TrialFunctions(W)
(v_r, v_i) = TestFunctions(W)

In [99]:
## Potential terms
V_e = Expression('4.0-1.0*(cos(k10*x[0]+k11*x[1]) + cos(k20*x[0]+k21*x[1]) + cos(k30*x[0]+k31*x[1]))', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)
W_o = Expression('sin(k10*x[0]+k11*x[1]) + sin(k20*x[0]+k21*x[1]) + sin(k30*x[0]+k31*x[1])', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)
V_e_2 = Expression('cos(k10*x[0]+k11*x[1]) + cos(k20*x[0]+k21*x[1]) + cos(k30*x[0]+k31*x[1])', \
                 k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)

delta = 1.0

C_r = as_matrix( ((V_e, 0.0), (0.0, V_e)) )
C_i = as_matrix( ((0.0, -delta*V_e_2), (delta*V_e_2, 0.0)) )

In [100]:
## Computations

# assemble mass matrix
m_r = (u_r*v_r + u_i*v_i)*dx
m_i = (-u_r*v_i + u_i*v_r)*dx
M = PETScMatrix()
assemble(m_r+m_i, tensor=M)

#L = inner(Constant(1.0), v_r+v_i)*dx
#b = PETScVector()
#assemble_system(m_r+m_i, L, A_tensor=M, b_tensor=b)

# eigenvalue solver
def compute_evals(A, E, M, l, r):
    
    # create eigensolver
    eigensolver = SLEPcEigenSolver(A,M)
    eigensolver.parameters['spectrum'] = 'smallest real'
    eigensolver.parameters['solver']   = 'krylov-schur'
    eigensolver.parameters['tolerance'] = 1.e-5   # 1.e-8
    #eigensolver.parameters['maximum_iterations'] = 1000
    #eigensolver.parameters['problem_type'] = 'gen_hermitian'


    # solve for eigenvalues
    eigensolver.solve(evals)
    
    for i in range(0,evals):
        if i <= (eigensolver.get_number_converged()-1):
            E[l][r][i] = eigensolver.get_eigenvalue(i)[0]
    
    #return np.sort(E)
    return E

program_starts = time.time()

r = 0

# main computation
for xi in SHIFT:
    
    k_vec = K+xi*kpar
    
    l = 0    
    for lam in LAMBDA:
    
        kk = k_vec + lam*ktilde
        k = Constant(kk)

        # define problem 
        a_r = ( inner(C_r*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_r*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) +\
               -inner(C_i*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) + inner(C_i*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) )*dx
        a_i = ( inner(C_r*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) - inner(C_r*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) +\
               +inner(C_i*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_i*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) )*dx

        # assemble stiffness matrix
        A = PETScMatrix()
        assemble(a_r+a_i, tensor=A)

        # assemble_system(a_r+a_i, L, bcs, A_tensor=A, b_tensor=b)

        E = compute_evals(A, E, M, l, r)
        
        l += 1

    r += 1
    print r/float(J)
    
print time.time() - program_starts

0.0322580645161
0.0645161290323
0.0967741935484
0.129032258065
0.161290322581
0.193548387097
0.225806451613
0.258064516129
0.290322580645
0.322580645161
0.354838709677
0.387096774194
0.41935483871
0.451612903226
0.483870967742
0.516129032258
0.548387096774
0.58064516129
0.612903225806
0.645161290323
0.677419354839
0.709677419355
0.741935483871
0.774193548387
0.806451612903
0.838709677419
0.870967741935
0.903225806452
0.935483870968
0.967741935484
1.0
1349.42062521


In [101]:
for x in range(0,3,1):
    Emin = [1e4]*J
    Emax = [0]*J
    for Eslice in E:
        for j in range(len(Eslice)):
            #newMin = min([e for e in Eslice[j] if e>0])
            if Eslice[j][x] == 0.0:
                continue
            if Eslice[j][x] < Emin[j]:
                Emin[j] = Eslice[j][x]
            if Eslice[j][x] > Emax[j]:
                Emax[j] = Eslice[j][x]
    print 'ymin =', Emin, ';'
    print ''
    print 'ymax =', Emax, ';'
    print ''
  
# MATLAB plotting
# y = [ymin,ymax,ymin(1)]
# x = [linspace(0,2*pi,11),linspace(2*pi,0,11),0]
# figure; fill(x,y,'k')
# hold on; fill(x,y,'k')

ymin = [0.032676787999459167, 0.11353883650733919, 0.2351066272404719, 0.46982533726991788, 0.81769320108881161, 1.2784065602887622, 1.8515592025797947, 2.5366329581183233, 3.2608821406671069, 4.095747578800399, 5.0406145054904163, 6.0944597316758404, 7.2560515436574997, 8.5239172713569609, 9.8257835237448035, 11.230470199569352, 9.8138528590194127, 8.4876646380140528, 7.2667398746304457, 6.1526886771641758, 5.0749662272630234, 4.106091274634319, 3.2471753388262172, 2.4990619651674431, 1.8624481771483354, 1.337901164689079, 0.85311968322594911, 0.48107235836826706, 0.22207447297835953, 0.076332668134416082, 0.043961399257298507] ;

ymax = [43.098884535782531, 42.804254109172383, 42.324197522120386, 41.713118512382778, 41.013799870467324, 40.258757488282527, 39.472560072879958, 38.674093565839705, 38.149763840944338, 37.638454422563782, 37.191323450205729, 36.778021115629542, 36.407699476123469, 36.062735119639967, 36.032058804709322, 35.900730114256703, 36.031125912611685, 36.203137056

In [102]:
E

array([[[  43.09888454,   73.32627237,   74.66178078,  233.96540062],
        [  42.53177128,   70.56504707,   78.56511256,  227.38062972],
        [  40.93515045,   68.56085221,   83.87353849,  217.42722633],
        ..., 
        [  40.55106052,   67.77104248,   83.39584439,  218.83483777],
        [  42.05706974,   70.02929319,   77.93903236,  228.7045185 ],
        [  42.60355106,   73.35750093,   73.50421902,  234.4113877 ]],

       [[  42.2724599 ,   70.63588419,   78.79328321,  228.70745843],
        [  42.80425411,   71.23753973,   77.21313772,  228.81440798],
        [  41.9893092 ,   70.2422101 ,   79.70835933,  226.93468463],
        ..., 
        [  37.66751628,   68.37493664,   88.30434985,  212.76294161],
        [  40.19599189,   68.90129807,   82.55190104,  222.31987759],
        [  41.89254161,   70.31191452,   77.86980995,  229.60416268]],

       [[  40.00054525,   69.78198902,   83.75303573,  222.62734005],
        [  41.86170011,   69.41563473,   80.79811716,  222